In [2]:
from collections import Counter, OrderedDict
from pathlib import Path
import sys
import textwrap
from tqdm import tqdm
import torch
from typing import Dict, List, Set, Union

from transformers import Seq2SeqTrainingArguments

import unicodedata
import uuid

from aic_nlp_utils.json import read_jsonl, read_json, write_json, write_jsonl
from aic_nlp_utils.fever import fever_detokenize, import_fever_corpus_from_sqlite
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
import stanza
# stanza.download("en")

sys.path.append('Claim_Generation')
from T5_QG import pipeline
from distractor_generation import Distractor_Generation

sys.path.append('Models')
from arguments import ModelArguments, DataTrainingArguments
from load import load_tokenizer_and_model, find_last_checkpoint

%load_ext autoreload
%autoreload 2

In [7]:
data = read_jsonl("/mnt/data/factcheck/qa2d/cs/dev.jsonl")

In [8]:
data[0]

{'question': 'Z čeho jsou vyrobeny pozoruhodné domy v Džajpuru?',
 'answer': 'druh pískovce s převládajícím růžovým odstínem',
 'turker_answer': 'pozoruhodné domy v Džajpuru z pískovce, kterému dominuje růžový odstín.',
 'rule-based': 'Pozoruhodné domy v Džajpuru jsou postaveny z pískovce, kterému dominuje růžový odstín.'}

In [3]:
model_args = ModelArguments(model_name_or_path="/home/drchajan/devel/python/FC/Zero-shot-Fact-Verification/experiments/qa2d/facebook/mbart-large-cc25_cs_CZ/BEST/checkpoint-26000")
tokenizer, model, data_collator = load_tokenizer_and_model(model_args, lang="cs_CZ", fp16=True)

In [11]:
model.to("cuda");

In [20]:
def predict(model, tokenizer, inputs, max_source_length=1024, padding=False):
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True, return_tensors="pt")
    model_inputs = {k: model_inputs[k].to("cuda") for k in model_inputs.keys()}
    with torch.no_grad():
        Y = model.generate(**model_inputs, max_new_tokens=768)
        predictions = tokenizer.batch_decode(
            Y, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
    return predictions


sample = data[6]
question = sample["question"]
answer = sample["answer"]
# question = "V kolika letech zemřel Petr?"
# answer = "25"
print(textwrap.fill(question))
print(answer)
predict(model, tokenizer, [answer + "</s>" + question])

Proč organismy dědí vlastnosti svých rodičů?
buňky potomků obsahují kopie genů z buněk jejich rodičů


['organismy dědí vlastnosti svých rodičů, protože buňky potomků obsahují kopie genů z buněk jejich rodičů.']